In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from langdetect import detect
from translate import Translator
import os

In [26]:
from googletrans import Translator
from langdetect import detect
import re
from bs4 import BeautifulSoup

translator = Translator()

def extract_reviews(soup):
    reviews = []
    review_divs = soup.find_all('div', {'data-hook': 'review'})

    for div in review_divs:
        try:
            review = {}
            review['customer_name'] = div.find('span', {'class': 'a-profile-name'}).text.strip()
            review['rating'] = div.find('span', {'class': 'a-icon-alt'}).text.split(' ')[0]
            
            a_tag = div.find('span', {'data-hook': 'review-title'})
            if a_tag:
                review['review_title'] = a_tag.find_all('span')[0].text.strip()
            else:
                review['review_title'] = ""
            
            text_element = div.find('span', {'class': 'a-size-base review-text review-text-content'})
            if text_element and text_element.span:
                text = text_element.span.text.strip()
                if text:
                    language = detect(text)
                    if language != 'en':
                        text = translator.translate(text, dest='en').text
                review['review_text'] = text
            else:
                review['review_text'] = ""
            
            review['review_date'] = div.find('span', {'data-hook': 'review-date'}).text.strip()
            pname = soup.find('a', {'data-hook': 'product-link'}).text.strip()
            review['product_name'] = re.sub(r',', '', pname)
            reviews.append(review)
        except Exception as e:
            print(f"An error occurred: {e}")
            continue  # Skip to the next review in case of an error

    print("-------------completed successfully---------------")
    return reviews


In [27]:
all_reviews = []

In [28]:
def scrape_reviews(url):
    headers = {
        'authority': 'www.amazon.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_reviews.extend(extract_reviews(soup))

In [29]:
def save_to_csv(reviews, filename='reviewsx.csv'):
    df = pd.DataFrame(reviews)
    if os.path.isfile(filename):
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, index=False)

In [30]:
import time

# Read URLs from the file
with open('urls.txt', 'r') as file:
    urls = file.readlines()

# Remove any extra whitespace (e.g., newline characters) from each URL
urls = [url.strip() for url in urls]

cnt = 0
# Loop through each URL
for current_url in urls:
    try:
        print("scrapping data from url : ",current_url)
        scrape_reviews(current_url)
        cnt = cnt +1
        if cnt==10:
            cnt = 0
            save_to_csv(all_reviews,"reviews.csv")
            all_reviews = []
        time.sleep(2)
    except Exception as e:
        print(f"An error occurred: {e}")
        break

save_to_csv(all_reviews,"reviews.csv")

scrapping data from url :  https://www.amazon.in/Intel-Core-i3-12100-Quad-core-Processor/product-reviews/B09NPHJLPT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
-------------completed successfully---------------
scrapping data from url :  https://www.amazon.in/Intel-Core-i3-12100-Quad-core-Processor/product-reviews/B09NPHJLPT/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2
-------------completed successfully---------------
scrapping data from url :  https://www.amazon.in/Intel-Core-i3-12100-Quad-core-Processor/product-reviews/B09NPHJLPT/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3
-------------completed successfully---------------
scrapping data from url :  https://www.amazon.in/Intel-Core-i3-12100-Quad-core-Processor/product-reviews/B09NPHJLPT/ref=cm_cr_getr_d_paging_btm_next_4?ie=UTF8&reviewerType=all_reviews&pageNumber=4
-------------completed successfully---------------
scrapping data from url :  ht